In [ ]:
# scenic analysis
# libraries
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import loompy as lp

sc.set_figure_params(dpi=150)
sns.set_style("ticks")

In [ ]:
# Load dataset, filter lowly expressed genes, standardize index names, and add "Gene" and "CellID" columns to metadata
adata = sc.read_loom("/path/to/folder/loom_converted_from_seurat.loom")

sc.pp.filter_genes(adata, min_cells=5)

adata.obs.index = list(adata.obs_names)
adata.var.index = list(adata.var_names)

adata.var["Gene"] = adata.var_names
adata.obs["CellID"] = adata.obs_names

adata.write_loom("/path/to/folder/new_loom.loom")

In [ ]:
# pyscenic grn step
!. "/opt/conda/etc/profile.d/conda.sh" && conda activate scenic && pyscenic grn \
  --num_workers 20 \
  -o  /path/to/folder/dataset_adjacencies.tsv \
  /path/to/folder/new_loom.loom\
  /path/to/folder/dataset/resources/mm_mgi_tfs.txt

In [ ]:
# pyscenic ctx step
!. "/opt/conda/etc/profile.d/conda.sh" && conda activate scenic && pyscenic ctx \
  /path/to/folder/dataset_adjacencies.tsv \
  /path/to/folder/resources/mm10__refseq-r80__10kb_up_and_down_tss.mc9nr.genes_vs_motifs.rankings.feather \
  /path/to/folder/resources/mm10__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings.feather \
  --annotations_fname /path/to/folder/resources/motifs-v9-nr.mgi-m0.001-o0.0.tbl \
  --expression_mtx_fname  /path/to/folder/new_loom.loom \
  --output /path/to/folder/dataset_regulon.tsv \
  --mode "dask_multiprocessing" \
  --num_workers 20

In [ ]:
# pyscenic aucell step
!. "/opt/conda/etc/profile.d/conda.sh" && conda activate scenic && pyscenic aucell \
  /path/to/folder/new_loom.loom\
  /path/to/folder/dataset_regulon.tsv \
  -o /path/to/folder/scenic.loom \
  --num_workers 20

In [ ]:
# Extract SCENIC regulon activities and gene targets from loom file
lf = lp.connect("/path/to/folder/scenic.loom", mode="r+", validate=False )

auc_mtx = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
scenic = sc.AnnData(auc_mtx, obs=adata.obs)
scenic.to_df().to_csv("/path/to/folder/activities.tsv", sep="\t")

regulons = lf.ra.Regulons
res = pd.concat([pd.Series(r.tolist(), index=regulons.dtype.names) for r in regulons], axis=1)
res.columns = lf.row_attrs["var_names"]
res.to_csv("/path/to/folder/mat.csv")
with open("/path/to/folder/genes.txt", "w") as f:
    f.write("".join([j + "\n" + i.strip(", ") + "\n\n" for i, j in zip((res.columns + ", ") @ res.astype(bool).values.T, res.index)]).strip())

lf.close()